In [9]:
from tensornetworks_pytorch.PosMPS import PosMPS

In [10]:
mps = PositiveMPS(D=2, d=5, verbose=True) 

In [11]:
list(mps.parameters())[1]

Parameter containing:
tensor([0., 0., 0., 0.], requires_grad=True)

In [12]:
mps.left_boundary.square()

tensor([0., 0., 0., 0.], grad_fn=<PowBackward0>)

In [13]:
import pickle
with open('datasets/lymphography', 'rb') as f:
        a=pickle.load(f)
X=a[0]
X=X.astype(int)

In [14]:
X

array([[2, 1, 0, ..., 1, 1, 0],
       [2, 3, 0, ..., 1, 1, 3],
       [1, 1, 1, ..., 1, 1, 0],
       ...,
       [2, 2, 0, ..., 1, 1, 2],
       [2, 3, 0, ..., 1, 1, 5],
       [1, 3, 1, ..., 0, 1, 0]])